# Cellpose

This notebook runs Cellpose segmentation on a folder of **.tif** files and outputs text outlines for ImageJ/FIJI to the folder containing the `.tif` images. Convert `.czi` images to `.tif` in ImageJ/FIJI first.

### Defining your image folder:

1. Go to the file browser system on your computer.
2. Navigate to the folder holding your images.
3. The image folder path is shown in the top bar. Rewrite this as anything following the "Data (D:)" with backslashes instead of arrows. Do not delete the leading "r'D:" Example:
    ```python
    r'D:\Monaghan-Lab\Example\Tif_Folder'
    ```
4. If you are on a Mac, your path would instead look like this:
    ```python
    r'/Monaghan-Lab/Example/Tif_Folder'
    ```

### Running this notebook:

1. **Edit path to images!!**
    - **Important note:** Ensure there are no quotation marks, spaces, or special characters in your folder names. Sometimes these are fine, but to prevent headaches it's best to just avoid using them :-)
    - Another note: you must define the **folder** containing the images; do not define the path to an individual file. In other words, no:
    ```python
    r'D:\Monaghan Lab\Example\Tif_Folder\Image.tif'
    ```

2. Edit the parameters in the main Cellpose block as needed. See the [documentation](https://cellpose.readthedocs.io/en/latest/) for help. I recommend running a few test images to make sure the parameters are working well for them. 

If you run into errors, first check that your image folder is defined correctly.
   - Note: ``` [WARNING] WARNING: not interp ``` in the output is not an error. The segmentation will continue.
   
The ``` _cp_outlines.txt ``` files generated below can be used in ImageJ/FIJI with the [ROI converter macro](https://github.com/MouseLand/cellpose/blob/master/imagej_roi_converter.py).

In [ ]:
## DO NOT EDIT ##
# Importing necessary packages
# You may have to install these with conda if you don't already have them in your environment
from pathlib import Path
import time, os, sys
from cellpose import utils, models, io

print('Done')

### Edit image folder here

In [ ]:
# File path to image folder
# Note: Do not delete the leading r

image_folder=Path(r'/path/to/images').glob('*.tif')

print('Done')

## Run Cellpose

Change the parameters in the loop where indicated and as needed. The main block will run for a while and continually update the output as it progresses; depending on how large they are, each image should only take a few minutes. Leave this browser window open and check back regularly. See [the cellpose documentation](https://cellpose.readthedocs.io/en/latest/) for more information.

#### Information for what can be edited:

##### Defining cellpose model #####

`model_type = 'cyto'` or `model_type = 'cyto2'` for cells; `model_type = 'nuclei'` for nuclei.

`gpu = True` to use GPU, `gpu = False` to not use GPU. GPU use is highly suggested by cellpose (and me) to speed up segmentation. If it's turned off it will slow down considerably, and the segmentation results may not be ideal.

##### Segmentation loop #####

`diameter = None` will estimate cell sizes on an image-by-image basis. Change to an integer (in pixels) to use the same diameter for each image. If diameter is too large, cells may be over-merged. If diameter is too small, cells may be incorrectly split up. This may take some trial and error to find a good value.

`channels=[a,b]` are the channels used. Edit this in `io.masks_flows_to_seg` too.
- The first element `a` is the channel to segment. The second element `b` is an optional nuclear channel. 
- `a`: `0` = grayscale, `1` = red, `2` = green, `3` = blue.
- `b`: `0` = none, otherwise the same color options as `a`.

`min_size` (not shown below) sets the minimum number of pixels per mask. Default is `15`, turn off using `min_size = -1`.

`flow_threshold` sets a threshold of error allowed between flows and masks. The default value is `0.4`. Increase if there are too few masks, but decrease if your masks are poorly shaped.

`mask_threshold` (not shown below) will discard masks below a certain probability. Default is `mask_threshold = 0.0` to keep all masks, while `mask_threshold = 1.0` will discard all. Too high, and there may not be as many areas covered as expected. Too low, and there will be too many masks from "dim" areas.

### In 3D

You have two main options for dealing with 3D images in cellpose: one which segments your 2D slices and stitches them together, and the other which runs segmentation volumetrically.

##### `do_3D=False`

Use this if volumetric segmentation is not working well.

`stitch_threshold`: 2D XY masks will stich across Z axis if the mask between the current and next slices are at or above the threshold value. This might take some trial and error to get quality stitching.

##### `do_3D=True`

`flow_threshold` is ignored in 3D. Use `min_size` instead to filter false positives.

`diameter = None` is ignored in 3D.

`anisotropy` accounts for sampling differences between XY and Z axes. For example, set to `2.0` if Z is sampled half as densely as XY.

`channel_axis` sets the image axis corresponding to the channels. If `channel_axis = None`, cellpose will automatically try to determine it.

`z_axis` sets the image axis corresponding to Z. If `z_axis = None`, cellpose will automatically try to determine which axis is Z.

In [ ]:
## Edit as necessary ##
## RUN CELLPOSE ##

# Defining cellpose model
model = models.Cellpose(gpu = True, model_type = 'cyto2')

# Segmentation Loop
for tif in image_folder:
    img = io.imread(tif)
    basename = os.path.splitext(tif.name)[0]
    
    # To troubleshoot - un-comment the print statement below to check that your loop
    # is going through the correct folder and finding the right images
    # print(tif)
    
    ## Edit parameters here ##
    
    masks, flows, styles, diams = model.eval(img,
                                             diameter = None,
                                             channels = [0,0],
                                             flow_threshold = 0.4,
                                             do_3D = False)
    
    ## Done editing ##
    
    # Save outlines as text for use in ROI converter macro
    outlines = utils.outlines_list(masks)
    io.outlines_to_text(os.path.splitext(tif)[0]+str(basename), outlines)
    
print('Segmentation Complete')